# Proyecto Final - Analisis De Datos

## Recopilación y Transformación de Datos (ETL)

Buscamos construir un modelo de predicción de churn (abandono de clientes), utilizando tanto información:

- Financiera y demográfica (edad, balance, salario, país, etc.),
- Macroeconómica (ingresos medios, desempleo, educación),
- De comportamiento digital (uso de la app, frecuencia de sesiones, tipo de dispositivo, etc.).

El **objetivo principal** es predecir si un cliente continuará o no con el producto/servicio de la empresa (variable Exited).

Este análisis permite:

- Identificar clientes en riesgo de abandonar
- Diseñar estrategias de retención personalizadas
- Optimizar campañas de fidelización
- Aumentar la rentabilidad reduciendo el churn rate.

### Importacion De Librerias

In [20]:
%pip install pandas openpyxl numpy matplotlib seaborn sqlite3

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql

### Leer Datos

1. Leemos los datos de `Churn_Modelling.csv` (csv)
    - Esta tabla contiene la información socioeconómica, financiera y de relación con la empresa de cada cliente.

In [22]:
df = pd.read_csv('./Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


2. Leemos los datos de los paises desde `country-data.xlsx` (Excel)
    - Esta base permite enriquecer la información del cliente con variables macroeconómicas y demográficas según su país, ofreciendo contexto sobre el entorno en el que vive.

In [23]:

countries = pd.read_excel('./country-data.xlsx')
countries.head()

,Country,AvgIncome_USD,UnemploymentRate_%,PopulationDensity_pkm2,EducationIndex,GDP_perCapita_USD
0,France,43000,7.5,119,0.88,46000
1,Spain,35000,12.2,94,0.84,38000
2,Germany,48000,5.6,233,0.91,52000
3,Italy,36000,9.8,200,0.85,41000
4,United Kingdom,45000,4.3,280,0.89,50000


3. Leemos los datos de comportamiento digital desde `digital_behavior.db` (Sqlite)
    - Esta base aporta información sobre el comportamiento digital del cliente, ayudando a entender su nivel de interacción con la plataforma.

In [24]:

conn = sql.connect('digital_behavior.db')

digital_behavior = pd.read_sql("SELECT * FROM DigitalBehavior;", conn)
digital_behavior.head()

,Identification,AvgSessionPerWeek,AvgSessionDuration_min,DaysSinceLastLogin,DeviceType,NumTransactionsApp,TimeOfDayMostActive,PushNotificationsEnabled,BiometricLoginUsed,DarkModeEnabled
0,15687492,1,9.1,81,Android,11,Night,0,1,0
1,15736963,4,6.5,78,iOS,8,Morning,1,0,1
2,15721730,3,12.7,57,Android,11,Afternoon,1,0,1
3,15762134,9,6.4,88,Web,13,Morning,0,1,1
4,15648898,4,9.6,47,iOS,11,Evening,0,0,1


## ETL

1. Relacionamos las diferentes fuentes de datos

In [ ]:
# Relacion con paises
df = df.merge(countries, left_on='Geography', right_on='Country', how='left')

df.drop(columns=['Country'], inplace=True) # Eliminar columna duplicada

df.head()

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Country,AvgIncome_USD,UnemploymentRate_%,PopulationDensity_pkm2,EducationIndex,GDP_perCapita_USD
0,1,15634602,Hargrave,619,Female,42,2,0.00,1,1,1,101348.88,1,France,43000,7.5,119,0.88,46000
1,2,15647311,Hill,608,Female,41,1,83807.86,1,0,1,112542.58,0,Spain,35000,12.2,94,0.84,38000
2,3,15619304,Onio,502,Female,42,8,159660.80,3,1,0,113931.57,1,France,43000,7.5,119,0.88,46000
3,4,15701354,Boni,699,Female,39,1,0.00,2,0,0,93826.63,0,France,43000,7.5,119,0.88,46000
4,5,15737888,Mitchell,850,Female,43,2,125510.82,1,1,1,79084.10,0,Spain,35000,12.2,94,0.84,38000


In [29]:
## Relacion con comportamiento digital
df = df.merge(digital_behavior, left_on="CustomerId", right_on="Identification", how="left")

df.drop(columns=['Identification'], inplace=True) # Eliminar columna duplicada

df.head()

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,...,Exited,AvgSessionPerWeek,AvgSessionDuration_min,DaysSinceLastLogin,DeviceType,NumTransactionsApp,TimeOfDayMostActive,PushNotificationsEnabled,BiometricLoginUsed,DarkModeEnabled
0,1,15634602,Hargrave,619,Female,42,2,0.00,1,1,...,1,4,1.5,72,iOS,9,Morning,1,1,1
1,2,15647311,Hill,608,Female,41,1,83807.86,1,0,...,0,4,8.3,53,Web,17,Morning,1,1,1
2,3,15619304,Onio,502,Female,42,8,159660.80,3,1,...,1,7,6.5,66,Android,10,Morning,1,1,0
3,4,15701354,Boni,699,Female,39,1,0.00,2,0,...,0,10,8.0,74,Web,12,Morning,1,0,1
4,5,15737888,Mitchell,850,Female,43,2,125510.82,1,1,...,0,6,8.1,45,Web,9,Night,1,0,0


2. Renombrar columnas para estandarizar

In [ ]:
df = df.rename(columns={
    "RowNumber": "row_number",
    "CustomerId": "customer_id",
    "Surname": "surname",
    "CreditScore": "credit_score",
    "Gender": "gender",
    "Age": "age",
    "Tenure": "tenure_years",
    "Balance": "account_balance",
    "NumOfProducts": "num_products",
    "HasCrCard": "has_credit_card",
    "IsActiveMember": "is_active_member",
    "EstimatedSalary": "estimated_salary",
    "Country": "country",
    "AvgIncome_USD": "avg_income_usd",
    "UnemploymentRate_%": "unemployment_rate_percent",
    "PopulationDensity_pkm2": "population_density_per_km2",
    "EducationIndex": "education_index",
    "GDP_perCapita_USD": "gdp_per_capita_usd",
    "AvgSessionPerWeek": "avg_sessions_per_week",
    "AvgSessionDuration_min": "avg_session_duration_min",
    "DaysSinceLastLogin": "days_since_last_login",
    "DeviceType": "device_type",
    "NumTransactionsApp": "num_transactions_app",
    "TimeOfDayMostActive": "most_active_time_of_day",
    "PushNotificationsEnabled": "push_notifications_enabled",
    "BiometricLoginUsed": "biometric_login_used",
    "DarkModeEnabled": "dark_mode_enabled",
    "Exited": "exited"
})

df.head()

,row_number,Id,surname,credit_score,gender,age,tenure_years,account_balance,num_products,has_credit_card,...,avg_sessions_per_week,avg_session_duration_min,days_since_last_login,device_type,num_transactions_app,most_active_time_of_day,push_notifications_enabled,biometric_login_used,dark_mode_enabled,exited
0,1,15634602,Hargrave,619,Female,42,2,0.00,1,1,...,4,1.5,72,iOS,9,Morning,1,1,1,1
1,2,15647311,Hill,608,Female,41,1,83807.86,1,0,...,4,8.3,53,Web,17,Morning,1,1,1,0
2,3,15619304,Onio,502,Female,42,8,159660.80,3,1,...,7,6.5,66,Android,10,Morning,1,1,0,1
3,4,15701354,Boni,699,Female,39,1,0.00,2,0,...,10,8.0,74,Web,12,Morning,1,0,1,0
4,5,15737888,Mitchell,850,Female,43,2,125510.82,1,1,...,6,8.1,45,Web,9,Night,1,0,0,0


3. Informacion de las columnas antes de transformarlas

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   row_number                  10000 non-null  int64  
 1   Id                          10000 non-null  int64  
 2   surname                     10000 non-null  object 
 3   credit_score                10000 non-null  int64  
 4   gender                      10000 non-null  object 
 5   age                         10000 non-null  int64  
 6   tenure_years                10000 non-null  int64  
 7   account_balance             10000 non-null  float64
 8   num_products                10000 non-null  int64  
 9   has_credit_card             10000 non-null  int64  
 10  is_active_member            10000 non-null  int64  
 11  estimated_salary            10000 non-null  float64
 12  country                     10000 non-null  object 
 13  avg_income_usd              1000

- Todos los valores estan completos, no hay nulos
- Hay valores de 0/1 como enteros

In [ ]:
# Fila de ejemplo
print(df.iloc[0])


row_number                            1
Id                             15634602
surname                        Hargrave
credit_score                        619
gender                           Female
age                                  42
tenure_years                          2
account_balance                     0.0
num_products                          1
has_credit_card                       1
is_active_member                      1
estimated_salary              101348.88
country                          France
avg_income_usd                    43000
unemployment_rate_percent           7.5
population_density_per_km2          119
education_index                    0.88
gdp_per_capita_usd                46000
avg_sessions_per_week                 4
avg_session_duration_min            1.5
days_since_last_login                72
device_type                         iOS
num_transactions_app                  9
most_active_time_of_day         Morning
push_notifications_enabled            1


4. Transformacion

In [ ]:
# Convertir columnas binarias (0/1) a Booleano
binary_cols = [
    "has_credit_card",
    "is_active_member",
    "push_notifications_enabled",
    "biometric_login_used",
    "dark_mode_enabled",
    "exited"
]
df[binary_cols] = df[binary_cols].astype(bool)

# Normalizar texto en columnas categóricas
df["gender"] = df["gender"].str.strip().str.capitalize()
df["country"] = df["country"].str.strip().str.title()
df["device_type"] = df["device_type"].str.strip().str.lower()
df["most_active_time_of_day"] = df["most_active_time_of_day"].str.strip().str.capitalize()

# Convertir columnas numéricas a tipo adecuado dtype
numeric_cols = [
    "credit_score", "age", "tenure_years", "account_balance", "num_products",
    "estimated_salary", "avg_income_usd", "unemployment_rate_percent",
    "population_density_per_km2", "education_index", "gdp_per_capita_usd",
    "avg_sessions_per_week", "avg_session_duration_min", "days_since_last_login",
    "num_transactions_app"
]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

# Crear columna que indique si el cliente tiene balance positivo
df["has_balance"] = df["account_balance"] > 0

# Crear columna por grupo de edad
df["age_group"] = pd.cut(
    df["age"],
    bins=[0, 25, 40, 60, 100],
    labels=["Youth", "Adult", "Middle-age", "Senior"]
)

# Crear columna por grupo de ingresos
df["income_group"] = pd.cut(
    df["avg_income_usd"],
    bins=[0, 30000, 60000, 100000, np.inf],
    labels=["Low", "Lower-Middle", "Upper-Middle", "High"]
)

# Convertir columnas de texto categóricas a tipo category
categorical_cols = [
    "gender",
    "country",
    "device_type",
    "most_active_time_of_day",
    "age_group"
]
df[categorical_cols] = df[categorical_cols].astype("category")

# Mover 'exited' al final
df['exited'] = df.pop('exited')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   row_number                  10000 non-null  int64   
 1   Id                          10000 non-null  int64   
 2   surname                     10000 non-null  object  
 3   credit_score                10000 non-null  int64   
 4   gender                      10000 non-null  category
 5   age                         10000 non-null  int64   
 6   tenure_years                10000 non-null  int64   
 7   account_balance             10000 non-null  float64 
 8   num_products                10000 non-null  int64   
 9   has_credit_card             10000 non-null  bool    
 10  is_active_member            10000 non-null  bool    
 11  estimated_salary            10000 non-null  float64 
 12  country                     10000 non-null  category
 13  avg_income_usd   

# Análisis Exploratorio de Datos (EDA)

blablabla

# Inteligencia de Negocios (BI)

blablabla

# Modelado Predictivo

blablabla

# Conclusiones y Recomendaciones

blablabla